# Passaggi preliminari

In [ ]:
# Importazione librerie
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

# Definizione funzioni

In [ ]:
# Funzione scorrimento pagina per caricare tutti i prodotti
def scroll_down(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(3)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
# Funzione ricerca prodotti e prezzi
def find_data(soup, cat):
    base = 'https://www.esselungaacasa.it/ecommerce/nav/auth/supermercato/home.html?streetId=226393'
    # Ricerca degli elementi desiderati (prezzi e nomi prodotti)
    desc  = soup.find_all('div', class_='description')
    prod  = soup.find_all('span', class_ = 'product-brand')
    # Creazione liste vuote
    prices   = []
    products = []
    category = []
    # Ricerca e salvataggio nomi prodotti
    for i in prod:
        # Nome prodotto
        text = i.get_text(strip=True)
        if len(text)>=55: # Condizione nome troppo lungo
            # RIcerca nodo parente
            a = i.find_parent('a')
            # Salvataggio link prodotto
            relativo = a.get('href')
            link = base + relativo
            time.sleep(3)
            # Switch e indirizzamento alla seconda pagina web
            driver.switch_to.window(driver.window_handles[1])
            driver.get(link)
            time.sleep(6)
            # RIcerca elementi desiderati salvando HTML con BeautifulSoup
            real_soup = BeautifulSoup(driver.page_source, 'html.parser')
            real_prod = real_soup.find('span', class_='product-brand')
            # Nome prodotto completo
            real_text = real_prod.get_text(strip=True)
            products.append(real_text)
            # Switch alla prima pagina web
            driver.switch_to.window(driver.window_handles[0])
        else:
            products.append(text) 
    # Ricerca e salvataggio prezzi   
    for element in desc:
        # Salvataggio prezzo
        price = element.find('p').text.strip()
        prices.append(price)
        # Salvo anche la categoria
        category.append(cat)
    # Costruzione output dataframe
    output = pd.DataFrame({'Category': category,
                           'Product': products,
                           'Price': prices})
    return output

In [ ]:
# Funzione accesso sito Esselunga
def access(driver):
    # Trovo l'elemento in cui inserire il CAP
    cap_input = driver.find_element(By.CSS_SELECTOR, "#postcode")
    # Inserisco un CAP
    cap_input.send_keys("20124")
    time.sleep(3)
    # Trovo l'elemento del pulsante "Verifica CAP" utilizzando il selettore CSS
    verifica_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
    # Click pulsante "Verifica CAP"
    verifica_button.click()
    time.sleep(6)
    # Trovo l'elemento del link "Continua come ospite"
    continua_ospite_link = driver.find_element(By.CSS_SELECTOR, "a[data-remodal-action='close']")
    # Click sul link "Continua come ospite"
    continua_ospite_link.click()
    time.sleep(6)
    return None

# Codice raccolta prodotti Esselunga

In [ ]:
# Apertura pagina chrome con webdriver
driver = webdriver.Chrome()
driver.execute_script('window.open()')
# URL di Esselunga
url_esselunga = 'https://www.esselungaacasa.it/ecommerce/nav/welcome/index.html'
# Indirizzamento alla pagina di Esselunga (è necessario fare il login con delle credenziali)
driver.get(url_esselunga)
time.sleep(3)
# Accesso alla pagina dei prodotti con inserimento di un CAP
access(driver)
# Trovo tutti i selettori delle varie categorie
categorie = driver.find_elements(By.CSS_SELECTOR, 'a[ng-click^="menuCtrl.open"] > span')
time.sleep(6)
# Salvo la lunghezza per poter fare un ciclo
categorie_len = len(categorie)
# Salvo i nomi delle categorie
nomi_categorie = [categoria.text for categoria in categorie]

In [ ]:
# Creazione di dataframe vuoto
Esselunga = pd.DataFrame({'Category': [],
                          'Product': [],
                          'Price': []})
# Ciclo for per iterare la raccolta prezzi su tutte le categorie
for elem in range(18,categorie_len-10): # Seleziono categorie di interesse
    # Click sulla categoria
    categorie[elem].click()
    time.sleep(6)
    # Caricamento di tutti i prodotti con la funzione scroll
    scroll_down(driver)
    time.sleep(3)
    # Salvataggio sorgente HTML e richiamo della funzione prezzi
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    temp = find_data(soup, nomi_categorie[elem])
    # Costruzione del dataset esselunga con il metodo .concat()
    Esselunga = pd.concat([Esselunga, temp], ignore_index=True)
    # Reindirizzamento alla pagina iniziale
    driver.get(url_esselunga)
    time.sleep(6)
    # Accesso alla pagina
    access(driver)
    # Ricerca categorie
    categorie = driver.find_elements(By.CSS_SELECTOR, 'a[ng-click^="menuCtrl.open"] > span')
    time.sleep(3)

In [ ]:
# Chiusura pagina web
driver.close()

In [ ]:
# Salvataggio dataframe in un file .cvs
Esselunga.to_csv("esselunga.csv")
Esselunga

,Category,Product,Price
0,SPESA BIO,"Esselunga Bio, cipolle dorate 500 g","Euro 3,16 / kg"
1,SPESA BIO,"Esselunga Bio, datterini biologici 300 g","Euro 8,27 / kg"
2,SPESA BIO,"Esselunga Bio, Insalatina 100 g","Euro 14,80 / kg"
3,SPESA BIO,"Esselunga Bio, 6 uova fresche biologiche","Euro 0,49 / pz"
4,SPESA BIO,"Esselunga Bio, prezzemolo tritato biologico su...","Euro 23,00 / kg"
...,...,...,...
15311,CURA CASA,vaschetta acciaio l30,"8,99 € / pz"
15312,CURA CASA,portatovaglioli metallo,"9,99 € / pz"
15313,CURA CASA,tovaglia antimacchia jungle 240cm,"18,90 € / pz"
15314,CURA CASA,Portafoto Rattan 13x18,"16,90 € / pz"
